In [1]:
import sys


no_of_cpu = 8
max_cores = 8

executor_mem = '56g'

Job_Name = 'Nishant : Cover_details'

print ("JOB NAME = "+str(Job_Name))
print ("No_of_cpu="+str(no_of_cpu))
print ("Max_cores="+str(max_cores))
print ("Executor_mem="+str(executor_mem))
# print ("Runtype="+str(runtype))
# print ("Report_date="+str(report_date))


from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import asc,lit
from pyspark.sql.functions import udf
#warnings.filterwarnings('error')
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F

conf = pyspark.SparkConf()
#import numpy
import calendar
#import pandas as pd
#import simplejson as json
#import pandas as pd
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from pyspark.sql.functions import lit
import simplejson as json
import json, pprint, requests
#es_nodes = '10.35.12.5'
#es_nodes = '10.35.12.6'
es_nodes = '10.35.12.212'
es_port = '9200'
es_user = 'elastic'
es_pwd = 'bEiilauM3es'
mesos_ip = 'mesos://10.33.195.18:5050'#'mesos://10.35.12.5:5050'
spark.stop() #############NEED TO COMMENT THIS SPARK.STOP WHEN RUNNING THROUGH SHELL###############################
conf.setMaster(mesos_ip)

conf.set('spark.executor.cores',no_of_cpu) ### change 1
conf.set('spark.executor.memory',executor_mem)
conf.set('spark.es.scroll.size','10000')
conf.set('spark.network.timeout','120m')
conf.set('spark.sql.crossJoin.enabled', 'true')
conf.set('spark.executor.heartbeatInterval','120s')
conf.set("spark.driver.cores","4")
conf.set("spark.driver.extraJavaOptions","-Xmx4g -Xms4g")
conf.set("spark.files.overwrite","true");
conf.set("spark.kryoserializer.buffer", "70");
conf.set("spark.driver.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.executor.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer");
conf.set("spark.broadcast.compress", "true");
conf.set("spark.shuffle.compress", "true");
conf.set("spark.shuffle.spill.compress", "true");
conf.set("spark.driver.maxResultSize", "4g");
conf.set("spark.app.name", Job_Name)
from pyspark.sql.functions import broadcast

conf.set('spark.driver.memory','12g') ### change 2
conf.set('spark.cores.max',max_cores) ### change 3
conf.set('spark.sql.shuffle.partitions','300')
conf.set('es.nodes.wan.only','true')
conf.set("spark.sql.autoBroadcastJoinThreshold",-1)
conf.set("spark.scroll.keepalive","120m")

conf.set('es.nodes',es_nodes)
conf.set('es.port',es_port)
conf.set('spark.es.net.http.auth.user', es_user)
conf.set('spark.es.net.http.auth.pass', es_pwd)

conf.set('spark.es.mapping.date.rich','false')
spark = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(spark)

# Load Data into PySpark DataFrames
# Prodcom Data Frame
import json, pprint, requests
import pyspark.sql.functions as sf
from pyspark.sql.types import *
from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import asc,lit
#warnings.filterwarnings('error')
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SQLContext
from pyspark.sql.window import Window
from pyspark import SparkContext, SparkConf
conf = pyspark.SparkConf()
#import numpy
import calendar
import time
#import pandas as pd
#import simplejson as json
#import pandas as pd
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType, StringType, DecimalType
from pyspark.sql.functions import lit
import simplejson as json
import json, pprint, requests
import re # to use regular expressions
import pyspark.sql.functions as F
from pyspark.sql import Window

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
19181,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

JOB NAME = Nishant : Cover_details
No_of_cpu=8
Max_cores=8
Executor_mem=56g

In [2]:

prod_url = "jdbc:postgresql://10.35.12.194:5432/gpadmin"
prod_host = '10.35.12.194'
prod_port = '5432'
prod_dbname = 'gpadmin'
user_prod="gpspark"
pwd_prod="spark@456"
dbschema="public"


prod_gpdb_spark_options ={
    "url": "jdbc:postgresql://{host}:{port}/{dbname}".format(host=prod_host,port=prod_port, dbname=prod_dbname),
    "user": "{user}".format(user=user_prod),
    "password": "{password}".format(password=pwd_prod)
} 

import psycopg2
import datetime
conn_prod = psycopg2.connect(host=prod_host, port=prod_port, user=user_prod, password=pwd_prod, dbname=prod_dbname)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:

def load_gpdb_jdbc(col_str,dbtable,dbschema,col_name=None,time_filter=None,partitionColumn="row_num"):
    """
    This is used to read gpdb with filter for columns and can apply other filter(date,values).
    Time filter contains startdate,enddate
    """
    gscPythonOptions = {
                        "url": prod_url,
                        "user": user_prod,
                        "password": pwd_prod,
                        "dbschema": dbschema,
                        "dbtable": dbtable,
                        "partitionColumn":partitionColumn,
                        "partitions": 8,
                        "server.port":"1150-1170"}
   
    if time_filter:
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates().filter(col(col_name).between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd')))
    else :
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates()
   
    return data

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
def get_latest_progress():
    
    try: 
        gscPythonOptions = {
                 "url":prod_url ,
                 "user":user_prod ,
                 "password": pwd_prod,
                 "dbschema":"customermart",
                 "dbtable": "progress",#table change
                 "server.port":"1150-1170"} 
        
        # this query will fetch till what date we have inserted the records in target
        last_run= sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
            .select('table_name','source','to_datetime').filter(col('table_name')=='policy_additional_details')\
            .filter(col('source')=='policy_additional_details123')
        
        Max_last_run =last_run.select(max('to_datetime')).first()[0]
        


        # if no record is available in progress then it will raise value error, in this case default value will be picked from except clause
        if Max_last_run is None:
            raise ValueError("No records found!")

        #print("Record Found. Progress updated till {}".format(Max_last_run))

    except Exception as E: 
        Max_last_run = datetime.datetime(1900,2,27, 0, 0)  # this is the default start date when no record is present in progress for customer_demographics table
        print("Executed_except",repr(E))
    return Max_last_run

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Generic function to save the progress from last run into Progress table
def update_progress(table_name,source,time_filter,records,start_time,starttime,status):
    output_index = "progress"
    schema = "customermart"


    import sys
    import time
    try:
        


        df_progress= sqlContext.createDataFrame([(table_name,source,
                                       time_filter['start_date'].strftime("%Y-%m-%d %H:%M:%S"),
                                         time_filter['end_date'].strftime("%Y-%m-%d %H:%M:%S"),records)]
                                     ,['table_name', 'source','from_datetime','to_datetime','records'])\
        .withColumn('start_time',lit(start_time)).withColumn('end_time',current_timestamp())\
                    .withColumn('executed_in_mins',lit((time.time() - starttime)/60))\
                                .withColumn('status',lit(status)).withColumn('remarks',lit(None))
    

        
        df_progress.write.format("greenplum")\
            .option("dbtable",output_index).option('dbschema',schema)\
            .option("server.port","1150-1170").option("url",prod_url)\
            .option("user", user_prod).option("password",pwd_prod).mode('append').save()
        
       
    except Exception as e:
        x = e
        print(x)
    else:
        x = 'success'
        print("Updated Progress for {table_name} from {source} until {to_datetime}. {records} records processed in this run".format(table_name=table_name, source=source,to_datetime=time_filter['end_date'].strftime("%Y-%m-%d %H:%M:%S"),records=records))
        print(x)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Define the Generic update function which takes the records currently in temp table (created with prefix 1 to original customercoe table)
# The function would first try to insert the records in original table. It that fails then it will upsert the records
# Since there is no direct upsert query an update and insert query with where clause is used
# This funciton just runs the SQL queries in GPDB and does not use spark

def update_policy(conn_to,table_name,time_filter,records,prod_gpdb_spark_options, schema="customermart"):
    
    conn_to.rollback()
#     gpdb_spark_options = prod_gpdb_spark_options
    primary_key = ['source_system_name','source_system_customer_id','policy_number']
    
    # these columns will be updated in case of duplicate records
    update_fields = ['intermediary_code','orphan_policy_indicator','orphan_policy_date','base_sum_insured','product_name',
                     'policy_end_date',"break_in_insurance_flag","break_in_insurance_days","ncb_flag","policy_term",
                     "policy_start_date","policy_renewal_count","ped_declared_indicator",
                     "portability_indicator","renewed_in_grace_period","vehicle_registration_no",'channel','premiumamount',
                     'total_premium','veh_model','veh_age','veh_make','veh_body','veh_engine_no' ,'veh_chassis_no',]
    
    # loading data from temp table to get column names    
    df_1 = sqlContext.read.format("greenplum").options(**prod_gpdb_spark_options).option("dbschema",schema)\
    .option("dbtable","{}_staging".format(table_name)).load()
    out_columns = df_1.columns
    out_columns = [col for col in out_columns if "json" not in col]

    cur_to = conn_to.cursor()  

    # this query insert records from temp table to target table when all the records in temp table are new and not present in target    
    only_insert_query = """insert into {schema}.{table_name} ({out_columns}) select {out_columns} from {schema}.\
    {table_name}_staging """.format(schema=schema,table_name=table_name,out_columns = ",".join(out_columns))
    
    # this query will be executed to update the records that are already present in target, new values will be assigned from temp table    
    update_query = """UPDATE {schema}.{table_name} orig
                          SET
                            {update_fields}
                          FROM
                            {schema}.{table_name}_staging temp
                           WHERE 
                            {primary_key}
                       """.format(schema=schema,table_name=table_name,primary_key= " and ".join([ "orig.{key} = temp.{key}".format(key=key) for key in primary_key]),update_fields = " , ".join([ "{key} = temp.{key}".format(key=key) for key in update_fields]))
    
    # this query will be executed after update query to insert the remaining records    
    insert_query = """ INSERT INTO {schema}.{table_name} ({out_columns})
                           SELECT {out_columns}
                           FROM
                             {schema}.{table_name}_staging temp
                           WHERE
                             NOT EXISTS (
                             SELECT 1 FROM {schema}.{table_name} orig WHERE 
                            {primary_key}
                            )
                        """.format(schema=schema,table_name=table_name,out_columns = ",".join(out_columns),primary_key = " and ".join([ "orig.{key} = temp.{key}".format(key=key) for key in primary_key]))

    try:
        print("Inside try segment")
        cur_to.execute(only_insert_query)
        print("Executed Only insert query")
#         update_progress(table_name,source,time_filter,records)
    except Exception as e:
        print("Inside except segment")
        conn_to.rollback()
        cur_to.execute(update_query)
        print("Executed update query")
        cur_to.execute(insert_query)
        print("Executed insert query")
#         update_progress(table_name,source,time_filter,records)
    conn_to.commit()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
import time
start_time = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
## added by mayur on 12 dec 2022 at 3:15 PM

from pyspark.sql.functions import hash
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
import psycopg2
import time
from time import gmtime, strftime
import pytz
starttime = time.time()
start_time = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S.%f")
start_time

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2024-03-21 07:44:28.061174'

In [ ]:
# Below code added by Mayur for taking latest till date on 12 dec-2022  

latest_progress=get_latest_progress()
# dates in string format
str_d1 = latest_progress.strftime("%Y-%m-%d")
str_d2 = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d")

# convert string to date object
d1 = datetime.datetime.strptime(str_d1, "%Y-%m-%d")
d2 = datetime.datetime.strptime(str_d2, "%Y-%m-%d")

# Executed_except ValueError('No records found!')
# {'start_date': datetime.datetime(2017, 1, 1, 0, 0), 'end_date': datetime.datetime(2024, 2, 27, 0, 0)}
# difference between dates in timedelta
delta = d2 - d1
day=delta.days-1  #currently T-2 change 2 to 1 for T-1

time_filter={'start_date':latest_progress,'end_date':latest_progress+datetime.timedelta(days=day)}
time_filter

In [ ]:
import json, pprint, requests
es_nodes = '10.35.12.212'
es_port = '9200'
es_user = 'elastic'
es_pwd = 'bEiilauM3es'

In [ ]:
# target_index = 'get_spa_additional_policy_details_2024-03-12'
# print(target_index)
# target_doc_type=target_index

# source =target_index
# doc_type = target_doc_type

# headers = {'Content-Type': 'application/json'}
# query = {'query':{'match_all':{}}}
# URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/'+source+'/_count'
# r = requests.get(URL, headers=headers)
# source_count222 = r.json().get('count')

# df_12 = sqlContext.read.format('org.elasticsearch.spark.sql')\
# .option('es.nodes' , es_nodes).option('es.port',es_port)\
# .option('es.resource',source)\
# .option('es.input.max.docs.per.partition',source_count222)\
# .load()

In [ ]:
col_str=['policy_number','certificate_no', 'pol_incept_date','pol_exp_date', 'source_system_customer_id', 'reference_num','sum_insured', 
         'premiumamount','total_premium','irda_product_uin','product_cd','lob',
         'renl_cert_no' , 'policy_counter','business_type' ,'txt_business_type','txt_intermediary_cd',  'commit_timestamp' ]

gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark",
"password": "spark@456",
"server.port": "1151-1170",
"dbschema":"customermart",
"dbtable": "policy_retail"}
pol = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str).drop_duplicates().filter(col('commit_timestamp').between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd'))).filter(col('lob')\
                                                                                               .isin('Mediseries',
'Covid Health',
'LTA',
'2W',
'Health Benefit',
'CVTP',
'Pvt Car',
'IPA',
'CV'))

In [ ]:
col_str=['num_reference_number','policy_no as previous_policy_no','policy_effective_to as previous_policy_exp_date', 'previous_policy_type','policy_effective_from']
pp=load_gpdb_jdbc(col_str,"policy_gc_previous_policy_details_tab",'public')

In [ ]:
col_str=['num_reference_no', 'application_no', 'policy_issuance_date','policy_no','certificate_no','rto_location', 'branch' ,'prdr_branch_sub_cd','customer_gstin','policy_type','zone','timestamp']
pr=load_gpdb_jdbc(col_str,"premium_register",'registers')

In [ ]:
pr=pr.withColumn('primary_key', concat_ws('',col('policy_no'),col('certificate_no'))).dropDuplicates()

In [ ]:
from pyspark.sql.window import Window

pr = pr.withColumn("rn", F.row_number()
        .over(Window.partitionBy("primary_key")
        .orderBy(F.col("timestamp").desc(),F.col("num_reference_no").desc())))

pr = pr.filter(F.col("rn") == 1).drop("rn")

In [ ]:
col_str=['dc_unified_id', 'source_customer_id']
mbdp=load_gpdb_jdbc(col_str,"mastercraft_bulk_detail_prod",'public')

In [ ]:
col_str=['dc_unified_id', 'source_customer_id','full_name','psnt_address_country','psnt_address_state','psnt_address_city','psnt_address_zip','dc_active_flag',
        'psnt_address_line_1','psnt_address_line_2','psnt_address_line_3','gstin', 'passport','pan','cleansed_passport', 'cleansed_pan', 'birth_date']
mmp=load_gpdb_jdbc(col_str,"mastercraft_master_prod",'public').filter(col('dc_active_flag')==1).drop('dc_active_flag')

In [ ]:
mmp=mmp.withColumn("birth_date",expr("case\
    when birth_date is not null and substring(birth_date,1,4) < '1922' then null\
    when birth_date is not null and substring(birth_date,1,4) > '2023' then null\
    when birth_date is not null then cast(concat(substring(birth_date,1,4),'-', substring(birth_date,6,2),'-', substring(birth_date,9,2)) as date)\
    else null end as birth_date")).alias("birth_date")

In [ ]:
mmp=mmp.withColumn("birth_date",when(col('birth_date')=='1990-01-01',None).otherwise(col('birth_date')))

In [ ]:
from pyspark.sql import functions as F
mmp = mmp.withColumn('age', (F.months_between(current_date(), F.col('birth_date')) / 12).cast('int'))

In [ ]:
col_str=['txt_intermediary_name as agent_name', 'txt_mobile','txt_telephone','txt_telephone1', 'txt_contact_phone','txt_emer_person_mobile_number',
         'txt_policy_phone','txt_alt_mobile', 'txt_license_no as agent_license_no', 'txt_intermediary_cd']
genmst=load_gpdb_jdbc(col_str,"reference_gc_genmst_intermediary",'public')

In [ ]:
genmst=genmst.withColumn('agent_mobile', coalesce('txt_mobile','txt_telephone','txt_telephone1', 'txt_contact_phone','txt_emer_person_mobile_number','txt_policy_phone','txt_alt_mobile'))

In [ ]:
df=pol.join(pr, pol.reference_num==pr.num_reference_no, 'inner').drop(pr.num_reference_no).drop(pr.policy_no).drop(pr.certificate_no)

In [ ]:
df=df.join(pp, df.reference_num==pp.num_reference_number, 'left').drop(pp.num_reference_number)

In [ ]:
df.columns

In [ ]:
from pyspark.sql.window import Window

df = df.withColumn("rn", F.row_number()
        .over(Window.partitionBy("policy_number",'certificate_no')
        .orderBy(F.col("timestamp").desc(),F.col("reference_num").desc())))

df = df.filter(F.col("rn") == 1).drop("rn").drop("primary_key")

In [ ]:
mc=mbdp.join(mmp,'dc_unified_id', 'left').drop(mmp.source_customer_id)

In [ ]:
# df=df.join(customer_gc_df, df.source_system_customer_id==customer_gc_df.customer_code, 'inner').drop(customer_gc_df.customer_code)
df=df.join(mc, df.source_system_customer_id==mc.source_customer_id, 'left').drop(mc.source_customer_id)
df=df.join(genmst,'txt_intermediary_cd','left')
df.columns

In [ ]:
df=df.withColumn('pan_new', coalesce('cleansed_pan', 'pan'))\
.withColumn('passport_new', coalesce('cleansed_passport', 'passport'))\
.withColumn('address', concat_ws('', 'psnt_address_line_1', 'psnt_address_line_2', 'psnt_address_line_3','psnt_address_city','psnt_address_state','psnt_address_country','psnt_address_zip'))

In [ ]:
df.columns

In [ ]:
df=df.withColumn('product_cd',lpad(col('product_cd'),6,'0'))

In [ ]:
col_str=['txt_endorsement_wording','num_reference_number','num_endorsement_no','txt_customer_id','num_net_od_premium','num_net_tp_premium',
         'dat_od_expiry_date','dat_od_start_date','dat_tp_expiry_date','dat_tp_start_date','num_covernote_no',
         'dat_cover_note_date','txt_cover_note_hour']
gen_prop=load_gpdb_jdbc(col_str,"policy_gc_gen_prop_information_tab",'public')

In [ ]:
df=df.join(gen_prop, df.reference_num==gen_prop.num_reference_number, 'left').drop(pp.num_reference_number)

In [ ]:
df.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
records=df.count()
records

# MOTOR

In [ ]:
df_motor=df.filter(col('lob').isin('2W',
'CV',
'CVTP',
'Pvt Car'))

In [ ]:
col_str=['information137','information288','information128','reference_num','information139','information126','risk1']
risk_headers=load_gpdb_jdbc(col_str,"policy_gc_risk_headers",'public')

In [ ]:
df_risk_header_pol=df_motor.join(risk_headers,'reference_num','left')

In [ ]:
df_risk_header_pol=df_risk_header_pol.withColumn('product_cd',when(col('product_cd').isin('3184','3187','3188','3189','3191'),lpad(col('product_cd'),6,'0'))\
                                                .otherwise(col('product_cd')))

In [ ]:
df_risk_header_pol=df_risk_header_pol.withColumn('mot_policy_plan_code',when(col('product_cd')=='003184',col('information137'))\
                                                .when(col('product_cd')=='003187',col('information288'))\
                                                 .when(col('product_cd')=='003188',col('information128'))\
                                                 .when(col('product_cd')=='003189',col('information128'))\
                                                .when(col('product_cd')=='003191',col('information128'))\
                                                .otherwise(lit('-')))\
.withColumn('mot_policy_plan',when(col('product_cd')=='003184',col('information139'))\
                                                .when(col('product_cd')=='003187',col('information139'))\
                                                 .when(col('product_cd')=='003188',col('information126'))\
                                                 .when(col('product_cd')=='003189',col('information126'))\
                                                .when(col('product_cd')=='003191',col('information126'))\
                                                .otherwise(lit('-')))\
.withColumn('mot_policy_variant',when(col('product_cd').isin('003184','003187','003188','003189','003191'),col('risk1'))\
                                                .otherwise(lit('-')))

In [ ]:
col_str=['reference_num','engine_number', 'chassis_number', 'cc','yr_of_manf','type_of_body','registered_seating_capacity', 'trailer_idv', 'owner_occupation' ,
         'totalidvy1', 'totalidvy2','totalidvy3','nonelectricalaccessoriesy1','nonelectricalaccessoriesy2', 'electricalaccessoriesidv','veh_idv',
         'nonelectricalaccessoriesy3','alternatepolicynumber','is_cnglpg_kit_pip','cng_lpg_si_tot','side_car_value','roadside_assistance',
        'noofclaimsdepreciationreimb','voluntary_deductible_amt','imposed_excess_amount','franchisee','compulsory_pa_ownerdriver','ncb_misdeclaration',
        'vehicle_seg', 'veh_type', 'vehicle_use', 'variant', 'vehicle_age', 'vehcile_subclass', 'vehicle_class','policy_plan']
mot=load_gpdb_jdbc(col_str,"policy_dh_risk_headers_mot",'public')

In [ ]:
col_str=['reference_num','make','model']
mot_risk=load_gpdb_jdbc(col_str,"policy_dh_risk_details_mot",'public')

In [ ]:
df_mot=df_risk_header_pol.join(mot, 'reference_num', 'left')
df_mot=df_mot.join(mot_risk, 'reference_num', 'left')
df_mot.columns

In [ ]:
df_distinct_mot=df_mot.withColumn('primary_key', concat_ws('',trim(col('source_system_customer_id')), trim(col('policy_number')),trim(col('reference_num')),
                                                       trim(col('certificate_no')))).dropDuplicates()

In [ ]:
from pyspark.sql.window import Window

df_distinct_mot = df_distinct_mot.withColumn("rn", F.row_number()
        .over(Window.partitionBy("primary_key")
        .orderBy(F.col("commit_timestamp").desc())))

df_distinct_mot = df_distinct_mot.filter(F.col("rn") == 1).drop("rn").drop("primary_key")

In [ ]:
df_distinct_mot=df_distinct_mot.withColumn('appointeename',lit(None).cast('string'))\
.withColumn('cum_discount_premium',lit(None).cast('string'))\
.withColumn('cum_loading_premium',lit(None).cast('string'))\
.withColumn('endt_discount_premium',lit(None).cast('string'))\
.withColumn('endt_loading_premium',lit(None).cast('string'))\
.withColumn('healthcheckup_limit',lit(None).cast('string'))\
.withColumn('insured_dob',lit(None).cast('string'))\
.withColumn('insured_name',lit(None).cast('string'))\
.withColumn('insuredage',lit(None).cast('string'))\
.withColumn('insuredname',lit(None).cast('string'))\
.withColumn('loading_reason_amhi',lit(None).cast('string'))\
.withColumn('member_id',lit(None).cast('string'))\
.withColumn('memberid',lit(None).cast('string'))\
.withColumn('nom_dob_hlt',lit(None).cast('string'))\
.withColumn('nomi_rel_hlt',lit(None).cast('string'))\
.withColumn('nominee_age',lit(None).cast('string'))\
.withColumn('nomineename_hlt',lit(None).cast('string'))\
.withColumn('outpatient_treatmentlimit',lit(None).cast('string'))\
.withColumn('preexisting_disease',lit(None).cast('string'))\
.withColumn('relationshipappnom',lit(None).cast('string'))\
.withColumn('dep_date',lit(None).cast('string'))\
.withColumn('dob',lit(None).cast('string'))\
.withColumn('id_no',lit(None).cast('string'))\
.withColumn('id_type',lit(None).cast('string'))\
.withColumn('insured_first_name',lit(None).cast('string'))\
.withColumn('insured_last_name',lit(None).cast('string'))\
.withColumn('insured_middle_name',lit(None).cast('string'))\
.withColumn('nominee_name',lit(None).cast('string'))\
.withColumn('rating_zone',lit(None).cast('string'))\
.withColumn('relationship',lit(None).cast('string'))\
.withColumn('return_date',lit(None).cast('string'))\
.withColumn('travel_duration',lit(None).cast('string'))\
.withColumn('insured_name',lit(None).cast('string'))






# .withColumn('appointeename',lit(None).cast('string'))\
# .withColumn('cum_discount_premium',lit(None).cast('string'))\
# .withColumn('cum_loading_premium',lit(None).cast('string'))\
# .withColumn('endt_discount_premium',lit(None).cast('string'))\
# .withColumn('endt_loading_premium',lit(None).cast('string'))\
# .withColumn('healthcheckup_limit',lit(None).cast('string'))\
# .withColumn('insured_dob',lit(None).cast('string'))\
# .withColumn('insured_name',lit(None).cast('string'))\
# .withColumn('insuredage',lit(None).cast('string'))\
# .withColumn('insuredname',lit(None).cast('string'))\
# .withColumn('loading_reason_amhi',lit(None).cast('string'))\
# .withColumn('member_id',lit(None).cast('string'))\
# .withColumn('memberid',lit(None).cast('string'))\
# .withColumn('nom_dob_hlt',lit(None).cast('string'))\
# .withColumn('nomi_rel_hlt',lit(None).cast('string'))\
# .withColumn('nominee_age',lit(None).cast('string'))\
# .withColumn('nomineename_hlt',lit(None).cast('string'))\
# .withColumn('outpatient_treatmentlimit',lit(None).cast('string'))\
# .withColumn('preexisting_disease',lit(None).cast('string'))\
# .withColumn('relationshipappnom',lit(None).cast('string'))\
# .withColumn('dat_cover_note_date',lit(None).cast('string'))\
# .withColumn('dat_od_expiry_date',lit(None).cast('string'))\
# .withColumn('dat_od_start_date',lit(None).cast('string'))\
# .withColumn('dat_tp_expiry_date',lit(None).cast('string'))\
# .withColumn('dat_tp_start_date',lit(None).cast('string'))\
# .withColumn('dep_date',lit(None).cast('string'))\
# .withColumn('insured_first_name',lit(None).cast('string'))\
# .withColumn('insured_last_name',lit(None).cast('string'))\
# .withColumn('insured_middle_name',lit(None).cast('string'))\
# .withColumn('insured_name',lit(None).cast('string'))\
# .withColumn('num_covernote_no',lit(None).cast('string'))\
# .withColumn('num_endorsement_no',lit(None).cast('string'))\
# .withColumn('num_net_od_premium',lit(None).cast('string'))\
# .withColumn('num_net_tp_premium',lit(None).cast('string'))\
# .withColumn('rating_zone',lit(None).cast('string'))\
# .withColumn('return_date',lit(None).cast('string'))\
# .withColumn('travel_duration',lit(None).cast('string'))\
# .withColumn('txt_cover_note_hour',lit(None).cast('string'))\
# .withColumn('txt_endorsement_wording',lit(None).cast('string'))


In [ ]:
df_distinct_mot.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
records=df_distinct_mot.count()
records

In [ ]:
df_distinct_mot=df_distinct_mot.select('reference_num', 'txt_intermediary_cd', 'policy_number', 'certificate_no', 'pol_incept_date', 'pol_exp_date', 'source_system_customer_id', 'sum_insured', 'premiumamount', 'total_premium', 'irda_product_uin', 'product_cd', 'lob', 'renl_cert_no', 'policy_counter', 'business_type', 'txt_business_type', 'commit_timestamp', 'application_no', 'policy_issuance_date', 'rto_location', 'branch', 'prdr_branch_sub_cd', 'customer_gstin', 'policy_type', 'zone', 'timestamp', 'previous_policy_no', 'previous_policy_exp_date', 'previous_policy_type', 'policy_effective_from', 'dc_unified_id', 'full_name', 'psnt_address_country', 'psnt_address_state', 'psnt_address_city', 'psnt_address_zip', 'psnt_address_line_1', 'psnt_address_line_2', 'psnt_address_line_3', 'gstin', 'passport', 'pan', 'cleansed_passport', 'cleansed_pan', 'birth_date', 'age', 'agent_name', 'txt_mobile', 'txt_telephone', 'txt_telephone1', 'txt_contact_phone', 'txt_emer_person_mobile_number', 'txt_policy_phone', 'txt_alt_mobile', 'agent_license_no', 'agent_mobile', 'pan_new', 'passport_new', 'address', 'txt_endorsement_wording', 'num_reference_number', 'num_endorsement_no', 'txt_customer_id', 'num_net_od_premium', 'num_net_tp_premium', 'dat_od_expiry_date', 'dat_od_start_date', 'dat_tp_expiry_date', 'dat_tp_start_date', 'num_covernote_no', 'dat_cover_note_date', 'txt_cover_note_hour','risk1', 'mot_policy_plan_code', 'mot_policy_plan', 'mot_policy_variant', 'engine_number', 'chassis_number', 'cc', 'yr_of_manf', 'type_of_body', 'registered_seating_capacity', 'trailer_idv', 'owner_occupation', 'totalidvy1', 'totalidvy2', 'totalidvy3', 'nonelectricalaccessoriesy1', 'nonelectricalaccessoriesy2', 'electricalaccessoriesidv', 'veh_idv', 'nonelectricalaccessoriesy3', 'alternatepolicynumber', 'is_cnglpg_kit_pip', 'cng_lpg_si_tot', 'side_car_value', 'roadside_assistance', 'noofclaimsdepreciationreimb', 'voluntary_deductible_amt', 'imposed_excess_amount', 'franchisee', 'compulsory_pa_ownerdriver', 'ncb_misdeclaration', 'vehicle_seg', 'veh_type', 'vehicle_use', 'variant', 'vehicle_age', 'vehcile_subclass', 'vehicle_class', 'policy_plan', 'make', 'model', 'appointeename', 'cum_discount_premium', 'cum_loading_premium', 'endt_discount_premium', 'endt_loading_premium', 'healthcheckup_limit', 'insured_dob', 'insured_name', 'insuredage', 'insuredname', 'loading_reason_amhi', 'member_id', 'memberid', 'nom_dob_hlt', 'nomi_rel_hlt', 'nominee_age', 'nomineename_hlt', 'outpatient_treatmentlimit', 'preexisting_disease', 'relationshipappnom', 'dep_date', 'dob', 'id_no', 'id_type', 'insured_first_name', 'insured_last_name', 'insured_middle_name', 'nominee_name', 'rating_zone', 'relationship', 'return_date', 'travel_duration')

In [ ]:
output_index = "mot_additional_details"
schema = "customermart"


import sys
try:

    df_distinct_mot.write.format("greenplum")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user", user_prod).option("password",pwd_prod).mode('overwrite').save()


except Exception as e:
    x = e
    print(x)
else:
    x = 200 #success
    print(x)

# Health

In [ ]:
df_health=df.filter(col('lob').isin('Mediseries',
'Covid Health',
'Health Benefit',
'IPA'))

In [ ]:
col_str=['reference_num', 'insured_name','insured_dob','preexisting_disease','member_id',
         'cum_loading_premium' ,'endt_loading_premium' ,'loading_reason_amhi' ,
'cum_discount_premium' ,'endt_discount_premium']
rd_hlt=load_gpdb_jdbc(col_str,"policy_dh_risk_details_hlt",'public')

In [ ]:
col_str=['healthcheckup_limit','reference_num','outpatient_treatmentlimit']
rh_hlt=load_gpdb_jdbc(col_str,"policy_dh_risk_headers_hlt",'public')

In [ ]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                    "user": "gpspark",
                    "password": "spark@456",
                    "dbschema": "public",
                    "dbtable": "underwriting_dh_cnfgtr_otherdt_grid_tab_hlt",
                    "partitionColumn":"row_num",
                    "partitions": 16,
                    "server.port": '1150-1170'}

uw_hlt = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                   .selectExpr("num_reference_number as reference_num", "nomineename as nomineename_hlt",'age as nominee_age','insuredage','insuredname',
                               'dateofbirth as nom_dob_hlt', "relationshipwithinsured as nomi_rel_hlt",'appointeename','relationshipappnom','memberid')\
                   .filter(col('nomineename_hlt').isNotNull())

uw_hlt.columns

In [ ]:
df_hlt=df_health.join(rd_hlt, df.reference_num==rd_hlt.reference_num, 'left').drop(rd_hlt.reference_num)
df_hlt=df_hlt.join(rh_hlt, df.reference_num==rh_hlt.reference_num, 'left').drop(rh_hlt.reference_num)
df_hlt=df_hlt.join(uw_hlt, df.reference_num==uw_hlt.reference_num, 'left').drop(uw_hlt.reference_num)

In [ ]:
df_distinct_hlt=df_hlt.withColumn('primary_key', concat_ws('',trim(col('source_system_customer_id')), trim(col('policy_number')),trim(col('reference_num')),
                                                       trim(col('certificate_no')),trim(col('memberid')))).dropDuplicates()

In [ ]:
df_distinct_hlt=df_distinct_hlt.drop_duplicates()

In [ ]:
from pyspark.sql.window import Window

df_distinct_hlt = df_distinct_hlt.withColumn("rn", F.row_number()
        .over(Window.partitionBy("primary_key")
        .orderBy(F.col("commit_timestamp").desc())))

df_distinct_hlt = df_distinct_hlt.filter(F.col("rn") == 1).drop("rn").drop("primary_key")

In [ ]:
df_distinct_hlt=df_distinct_hlt.withColumn('dep_date',lit(None).cast('string'))\
.withColumn('dob',lit(None).cast('string'))\
.withColumn('id_no',lit(None).cast('string'))\
.withColumn('id_type',lit(None).cast('string'))\
.withColumn('insured_first_name',lit(None).cast('string'))\
.withColumn('insured_last_name',lit(None).cast('string'))\
.withColumn('insured_middle_name',lit(None).cast('string'))\
.withColumn('nominee_name',lit(None).cast('string'))\
.withColumn('rating_zone',lit(None).cast('string'))\
.withColumn('relationship',lit(None).cast('string'))\
.withColumn('return_date',lit(None).cast('string'))\
.withColumn('travel_duration',lit(None).cast('string'))\
.withColumn('alternatepolicynumber',lit(None).cast('string'))\
.withColumn('cc',lit(None).cast('string'))\
.withColumn('chassis_number',lit(None).cast('string'))\
.withColumn('cng_lpg_si_tot',lit(None).cast('string'))\
.withColumn('compulsory_pa_ownerdriver',lit(None).cast('string'))\
.withColumn('electricalaccessoriesidv',lit(None).cast('string'))\
.withColumn('engine_number',lit(None).cast('string'))\
.withColumn('franchisee',lit(None).cast('string'))\
.withColumn('imposed_excess_amount',lit(None).cast('string'))\
.withColumn('is_cnglpg_kit_pip',lit(None).cast('string'))\
.withColumn('make',lit(None).cast('string'))\
.withColumn('model',lit(None).cast('string'))\
.withColumn('mot_policy_plan',lit(None).cast('string'))\
.withColumn('mot_policy_plan_code',lit(None).cast('string'))\
.withColumn('mot_policy_variant',lit(None).cast('string'))\
.withColumn('ncb_misdeclaration',lit(None).cast('string'))\
.withColumn('nonelectricalaccessoriesy1',lit(None).cast('string'))\
.withColumn('nonelectricalaccessoriesy2',lit(None).cast('string'))\
.withColumn('nonelectricalaccessoriesy3',lit(None).cast('string'))\
.withColumn('noofclaimsdepreciationreimb',lit(None).cast('string'))\
.withColumn('owner_occupation',lit(None).cast('string'))\
.withColumn('policy_plan',lit(None).cast('string'))\
.withColumn('registered_seating_capacity',lit(None).cast('string'))\
.withColumn('risk1',lit(None).cast('string'))\
.withColumn('roadside_assistance',lit(None).cast('string'))\
.withColumn('side_car_value',lit(None).cast('string'))\
.withColumn('totalidvy1',lit(None).cast('string'))\
.withColumn('totalidvy2',lit(None).cast('string'))\
.withColumn('totalidvy3',lit(None).cast('string'))\
.withColumn('trailer_idv',lit(None).cast('string'))\
.withColumn('type_of_body',lit(None).cast('string'))\
.withColumn('variant',lit(None).cast('string'))\
.withColumn('veh_idv',lit(None).cast('string'))\
.withColumn('veh_type',lit(None).cast('string'))\
.withColumn('vehcile_subclass',lit(None).cast('string'))\
.withColumn('vehicle_age',lit(None).cast('string'))\
.withColumn('vehicle_class',lit(None).cast('string'))\
.withColumn('vehicle_seg',lit(None).cast('string'))\
.withColumn('vehicle_use',lit(None).cast('string'))\
.withColumn('voluntary_deductible_amt',lit(None).cast('string'))\
.withColumn('yr_of_manf',lit(None).cast('string'))






# withColumn('alternatepolicynumber',lit(None).cast('string'))\
# .withColumn('cc',lit(None).cast('string'))\
# .withColumn('chassis_number',lit(None).cast('string'))\
# .withColumn('cng_lpg_si_tot',lit(None).cast('string'))\
# .withColumn('compulsory_pa_ownerdriver',lit(None).cast('string'))\
# .withColumn('electricalaccessoriesidv',lit(None).cast('string'))\
# .withColumn('engine_number',lit(None).cast('string'))\
# .withColumn('franchisee',lit(None).cast('string'))\
# .withColumn('imposed_excess_amount',lit(None).cast('string'))\
# .withColumn('is_cnglpg_kit_pip',lit(None).cast('string'))\
# .withColumn('make',lit(None).cast('string'))\
# .withColumn('model',lit(None).cast('string'))\
# .withColumn('mot_policy_plan',lit(None).cast('string'))\
# .withColumn('mot_policy_plan_code',lit(None).cast('string'))\
# .withColumn('mot_policy_variant',lit(None).cast('string'))\
# .withColumn('ncb_misdeclaration',lit(None).cast('string'))\
# .withColumn('nonelectricalaccessoriesy1',lit(None).cast('string'))\
# .withColumn('nonelectricalaccessoriesy2',lit(None).cast('string'))\
# .withColumn('nonelectricalaccessoriesy3',lit(None).cast('string'))\
# .withColumn('noofclaimsdepreciationreimb',lit(None).cast('string'))\
# .withColumn('owner_occupation',lit(None).cast('string'))\
# .withColumn('policy_plan',lit(None).cast('string'))\
# .withColumn('registered_seating_capacity',lit(None).cast('string'))\
# .withColumn('risk1',lit(None).cast('string'))\
# .withColumn('roadside_assistance',lit(None).cast('string'))\
# .withColumn('side_car_value',lit(None).cast('string'))\
# .withColumn('totalidvy1',lit(None).cast('string'))\
# .withColumn('totalidvy2',lit(None).cast('string'))\
# .withColumn('totalidvy3',lit(None).cast('string'))\
# .withColumn('trailer_idv',lit(None).cast('string'))\
# .withColumn('type_of_body',lit(None).cast('string'))\
# .withColumn('variant',lit(None).cast('string'))\
# .withColumn('veh_idv',lit(None).cast('string'))\
# .withColumn('veh_type',lit(None).cast('string'))\
# .withColumn('vehcile_subclass',lit(None).cast('string'))\
# .withColumn('vehicle_age',lit(None).cast('string'))\
# .withColumn('vehicle_class',lit(None).cast('string'))\
# .withColumn('vehicle_seg',lit(None).cast('string'))\
# .withColumn('vehicle_use',lit(None).cast('string'))\
# .withColumn('voluntary_deductible_amt',lit(None).cast('string'))\
# .withColumn('yr_of_manf',lit(None).cast('string'))\
# .withColumn('dat_cover_note_date',lit(None).cast('string'))\
# .withColumn('dat_od_expiry_date',lit(None).cast('string'))\
# .withColumn('dat_od_start_date',lit(None).cast('string'))\
# .withColumn('dat_tp_expiry_date',lit(None).cast('string'))\
# .withColumn('dat_tp_start_date',lit(None).cast('string'))\
# .withColumn('dep_date',lit(None).cast('string'))\
# .withColumn('lta_dob',lit(None).cast('string'))\
# .withColumn('lta_id_no',lit(None).cast('string'))\
# .withColumn('lta_id_type',lit(None).cast('string'))\
# .withColumn('insured_first_name',lit(None).cast('string'))\
# .withColumn('insured_last_name',lit(None).cast('string'))\
# .withColumn('insured_middle_name',lit(None).cast('string'))\
# .withColumn('num_covernote_no',lit(None).cast('string'))\
# .withColumn('num_endorsement_no',lit(None).cast('string'))\
# .withColumn('num_net_od_premium',lit(None).cast('string'))\
# .withColumn('num_net_tp_premium',lit(None).cast('string'))\
# .withColumn('rating_zone',lit(None).cast('string'))\
# .withColumn('return_date',lit(None).cast('string'))\
# .withColumn('travel_duration',lit(None).cast('string'))\
# .withColumn('txt_cover_note_hour',lit(None).cast('string'))\
# .withColumn('txt_endorsement_wording',lit(None).cast('string'))


In [ ]:
df_distinct_hlt.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
records=df_distinct_hlt.count()
records

In [ ]:
df_distinct_hlt=df_distinct_hlt.select(df_distinct_mot.columns)

In [ ]:
output_index = "hlt_additional_details"
schema = "customermart"


import sys
try:

    df_distinct_hlt.write.option("truncate", "true").format("greenplum")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user", user_prod).option("password",pwd_prod).mode('overwrite').save()


except Exception as e:
    x = e
    print(x)
else:
    x = 200 #success
    print(x)

# LTA

In [ ]:
df_lta=df.filter(col('lob').isin('LTA'))

In [ ]:
col_str=['dep_date', 'return_date',"ZONE as rating_zone",'travel_duration','reference_num']
rh_lta=load_gpdb_jdbc(col_str,"policy_dh_risk_headers_lta",'public')

In [ ]:
col_str=['reference_num','dob','id_no','id_type','insured_first_name','insured_middle_name','insured_last_name','nominee_name','relationship']
rd_lta=load_gpdb_jdbc(col_str,"policy_dh_risk_details_lta",'public')

In [ ]:
rd_lta=rd_lta.withColumn('insured_name', concat_ws(' ', 'insured_first_name','insured_middle_name','insured_last_name'))

In [ ]:
df_lta=df_lta.join(rh_lta, 'reference_num', 'left')
df_lta=df_lta.join(rd_lta,'reference_num', 'left')
df_lta.columns

In [ ]:
df_distinct_lta=df_lta.withColumn('primary_key', concat_ws('',trim(col('source_system_customer_id')), trim(col('policy_number')),trim(col('reference_num')),
                                                       trim(col('certificate_no')),col('insured_name'),col('id_no'))).dropDuplicates()

In [ ]:
from pyspark.sql.window import Window

df_distinct_lta = df_distinct_lta.withColumn("rn", F.row_number()
        .over(Window.partitionBy("primary_key")
        .orderBy(F.col("commit_timestamp").desc())))

df_distinct_lta = df_distinct_lta.filter(F.col("rn") == 1).drop("rn").drop("primary_key")

In [ ]:
df_distinct_lta.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
records=df_distinct_lta.count()
records

In [ ]:
df_distinct_hlt.unpersist()

In [ ]:
output_index = "lta_additional_details"
schema = "customermart"


import sys
try:

    df_distinct_lta.write.option("truncate", "true").format("greenplum")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user", user_prod).option("password",pwd_prod).mode('overwrite').save()


except Exception as e:
    x = e
    print(x)
else:
    x = 200 #success
    print(x)

In [ ]:
df_distinct_lta=df_distinct_lta.withColumn('alternatepolicynumber',lit(None).cast('string'))\
.withColumn('cc',lit(None).cast('string'))\
.withColumn('chassis_number',lit(None).cast('string'))\
.withColumn('cng_lpg_si_tot',lit(None).cast('string'))\
.withColumn('compulsory_pa_ownerdriver',lit(None).cast('string'))\
.withColumn('electricalaccessoriesidv',lit(None).cast('string'))\
.withColumn('engine_number',lit(None).cast('string'))\
.withColumn('franchisee',lit(None).cast('string'))\
.withColumn('imposed_excess_amount',lit(None).cast('string'))\
.withColumn('is_cnglpg_kit_pip',lit(None).cast('string'))\
.withColumn('make',lit(None).cast('string'))\
.withColumn('model',lit(None).cast('string'))\
.withColumn('mot_policy_plan',lit(None).cast('string'))\
.withColumn('mot_policy_plan_code',lit(None).cast('string'))\
.withColumn('mot_policy_variant',lit(None).cast('string'))\
.withColumn('ncb_misdeclaration',lit(None).cast('string'))\
.withColumn('nonelectricalaccessoriesy1',lit(None).cast('string'))\
.withColumn('nonelectricalaccessoriesy2',lit(None).cast('string'))\
.withColumn('nonelectricalaccessoriesy3',lit(None).cast('string'))\
.withColumn('noofclaimsdepreciationreimb',lit(None).cast('string'))\
.withColumn('owner_occupation',lit(None).cast('string'))\
.withColumn('policy_plan',lit(None).cast('string'))\
.withColumn('registered_seating_capacity',lit(None).cast('string'))\
.withColumn('risk1',lit(None).cast('string'))\
.withColumn('roadside_assistance',lit(None).cast('string'))\
.withColumn('side_car_value',lit(None).cast('string'))\
.withColumn('totalidvy1',lit(None).cast('string'))\
.withColumn('totalidvy2',lit(None).cast('string'))\
.withColumn('totalidvy3',lit(None).cast('string'))\
.withColumn('trailer_idv',lit(None).cast('string'))\
.withColumn('type_of_body',lit(None).cast('string'))\
.withColumn('variant',lit(None).cast('string'))\
.withColumn('veh_idv',lit(None).cast('string'))\
.withColumn('veh_type',lit(None).cast('string'))\
.withColumn('vehcile_subclass',lit(None).cast('string'))\
.withColumn('vehicle_age',lit(None).cast('string'))\
.withColumn('vehicle_class',lit(None).cast('string'))\
.withColumn('vehicle_seg',lit(None).cast('string'))\
.withColumn('vehicle_use',lit(None).cast('string'))\
.withColumn('voluntary_deductible_amt',lit(None).cast('string'))\
.withColumn('yr_of_manf',lit(None).cast('string'))\
.withColumn('appointeename',lit(None).cast('string'))\
.withColumn('cum_discount_premium',lit(None).cast('string'))\
.withColumn('cum_loading_premium',lit(None).cast('string'))\
.withColumn('endt_discount_premium',lit(None).cast('string'))\
.withColumn('endt_loading_premium',lit(None).cast('string'))\
.withColumn('healthcheckup_limit',lit(None).cast('string'))\
.withColumn('insured_dob',lit(None).cast('string'))\
.withColumn('insuredage',lit(None).cast('string'))\
.withColumn('insuredname',lit(None).cast('string'))\
.withColumn('loading_reason_amhi',lit(None).cast('string'))\
.withColumn('member_id',lit(None).cast('string'))\
.withColumn('memberid',lit(None).cast('string'))\
.withColumn('nom_dob_hlt',lit(None).cast('string'))\
.withColumn('nomi_rel_hlt',lit(None).cast('string'))\
.withColumn('nominee_age',lit(None).cast('string'))\
.withColumn('nomineename_hlt',lit(None).cast('string'))\
.withColumn('outpatient_treatmentlimit',lit(None).cast('string'))\
.withColumn('preexisting_disease',lit(None).cast('string'))\
.withColumn('relationshipappnom',lit(None).cast('string'))




# .withColumn('alternatepolicynumber',lit(None).cast('string'))\
# .withColumn('cc',lit(None).cast('string'))\
# .withColumn('chassis_number',lit(None).cast('string'))\
# .withColumn('cng_lpg_si_tot',lit(None).cast('string'))\
# .withColumn('compulsory_pa_ownerdriver',lit(None).cast('string'))\
# .withColumn('electricalaccessoriesidv',lit(None).cast('string'))\
# .withColumn('engine_number',lit(None).cast('string'))\
# .withColumn('franchisee',lit(None).cast('string'))\
# .withColumn('imposed_excess_amount',lit(None).cast('string'))\
# .withColumn('is_cnglpg_kit_pip',lit(None).cast('string'))\
# .withColumn('make',lit(None).cast('string'))\
# .withColumn('model',lit(None).cast('string'))\
# .withColumn('mot_policy_plan',lit(None).cast('string'))\
# .withColumn('mot_policy_plan_code',lit(None).cast('string'))\
# .withColumn('mot_policy_variant',lit(None).cast('string'))\
# .withColumn('ncb_misdeclaration',lit(None).cast('string'))\
# .withColumn('nonelectricalaccessoriesy1',lit(None).cast('string'))\
# .withColumn('nonelectricalaccessoriesy2',lit(None).cast('string'))\
# .withColumn('nonelectricalaccessoriesy3',lit(None).cast('string'))\
# .withColumn('noofclaimsdepreciationreimb',lit(None).cast('string'))\
# .withColumn('owner_occupation',lit(None).cast('string'))\
# .withColumn('policy_plan',lit(None).cast('string'))\
# .withColumn('registered_seating_capacity',lit(None).cast('string'))\
# .withColumn('risk1',lit(None).cast('string'))\
# .withColumn('roadside_assistance',lit(None).cast('string'))\
# .withColumn('side_car_value',lit(None).cast('string'))\
# .withColumn('totalidvy1',lit(None).cast('string'))\
# .withColumn('totalidvy2',lit(None).cast('string'))\
# .withColumn('totalidvy3',lit(None).cast('string'))\
# .withColumn('trailer_idv',lit(None).cast('string'))\
# .withColumn('type_of_body',lit(None).cast('string'))\
# .withColumn('variant',lit(None).cast('string'))\
# .withColumn('veh_idv',lit(None).cast('string'))\
# .withColumn('veh_type',lit(None).cast('string'))\
# .withColumn('vehcile_subclass',lit(None).cast('string'))\
# .withColumn('vehicle_age',lit(None).cast('string'))\
# .withColumn('vehicle_class',lit(None).cast('string'))\
# .withColumn('vehicle_seg',lit(None).cast('string'))\
# .withColumn('vehicle_use',lit(None).cast('string'))\
# .withColumn('voluntary_deductible_amt',lit(None).cast('string'))\
# .withColumn('yr_of_manf',lit(None).cast('string'))\
# .withColumn('appointeename',lit(None).cast('string'))\
# .withColumn('cum_discount_premium',lit(None).cast('string'))\
# .withColumn('cum_loading_premium',lit(None).cast('string'))\
# .withColumn('endt_discount_premium',lit(None).cast('string'))\
# .withColumn('endt_loading_premium',lit(None).cast('string'))\
# .withColumn('healthcheckup_limit',lit(None).cast('string'))\
# .withColumn('insured_dob',lit(None).cast('string'))\
# .withColumn('insuredage',lit(None).cast('string'))\
# .withColumn('insuredname',lit(None).cast('string'))\
# .withColumn('loading_reason_amhi',lit(None).cast('string'))\
# .withColumn('member_id',lit(None).cast('string'))\
# .withColumn('memberid',lit(None).cast('string'))\
# .withColumn('nom_dob_hlt',lit(None).cast('string'))\
# .withColumn('nomi_rel_hlt',lit(None).cast('string'))\
# .withColumn('nominee_age',lit(None).cast('string'))\
# .withColumn('nomineename_hlt',lit(None).cast('string'))\
# .withColumn('outpatient_treatmentlimit',lit(None).cast('string'))\
# .withColumn('preexisting_disease',lit(None).cast('string'))\
# .withColumn('relationshipappnom',lit(None).cast('string'))

In [ ]:
df_distinct_lta=df_distinct_lta.select(df_distinct_mot.columns)

# Main

In [ ]:
df_main=df_distinct_mot.union(df_distinct_hlt).union(df_distinct_lta)

In [ ]:
df_main.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
records=df_main.count()
records

In [ ]:
output_index = "policy_additional_details"
schema = "customermart"


import sys
try:

    df_main.write.option("truncate", "true").format("greenplum")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user", user_prod).option("password",pwd_prod).mode('append').save()


except Exception as e:
    x = e
    print(x)
else:
    x = 200 #success
    print(x)

In [ ]:
gscPythonOptions = {
         "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpcustcoe","password": "gpcustcoe@123",
    "dbschema":"customermart"
    ,"dbtable": "policy_additional_details",
         "server.port":"1150-1170"}

df_main = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()

In [ ]:
report_date = datetime.date.today()- datetime.timedelta(days=2)
report_date_1=report_date
report_date_str = str(report_date)

In [ ]:
report_date_str

In [ ]:
target_index=f'get_spa_additional_policy_details_{report_date_str}'
print(target_index)

In [ ]:
import json, pprint, requests
es_nodes = '10.35.12.212'
es_port = '9200'
es_user = 'elastic'
es_pwd = 'bEiilauM3es'

######target_index = 'actuarial_premium_risk_tables'----previous name...delete later

#alias_name = 'nepuepr_uat_3'
target_doc_type = f'get_spa_additional_policy_details_{report_date_str}'
#########delete index if already exists########################################################################################################################################################################

In [ ]:
# #########delete index ###############################################
# headers = {'Content-Type': 'application/json'}
# query = {"query":{"match_all":{}}}
# URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/'+target_index
# r = requests.delete(URL, data=json.dumps(query), headers=headers) 
# print(r)

In [ ]:
#########create index ##################################################################################################################################################################
headers = {'Content-Type': 'application/json'}
query_create= {"settings": { "index.number_of_shards": "1" }}
URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/'+target_index
r = requests.put(URL, data=json.dumps(query_create), headers=headers)
print(r)

In [ ]:
target_index_doc_type = target_index +'/'+ target_doc_type

In [ ]:
target_index_doc_type

In [ ]:
df_main=df_main.drop_duplicates()

In [ ]:
df_main.columns

In [ ]:
# from pyspark.sql.window import Window

# w = Window.partitionBy(['policy_number','certificate_no','member_id'])\
# .orderBy([F.col('commit_timestamp').desc(),F.col('reference_num').desc()])
# df_main = df_main.withColumn('row',F.row_number().over(w))
# df_main = df_main.filter(F.col('row')==1).drop(df_main.row).drop_duplicates()

In [ ]:
df_main=df_main.withColumn('policy_issuance_date',date_format(to_utc_timestamp(col('policy_issuance_date'),"UTC"),'yyyy-MM-dd'))\
.withColumn('pol_incept_date',date_format(to_utc_timestamp(col('pol_incept_date'),"UTC"),'yyyy-MM-dd'))\
.withColumn('pol_exp_date',date_format(to_utc_timestamp(col('pol_exp_date'),"UTC"),'yyyy-MM-dd'))\
.withColumn('birth_date',date_format(to_utc_timestamp(col('birth_date'),"UTC"),'yyyy-MM-dd'))\
.withColumn('previous_policy_exp_date',date_format(to_utc_timestamp(col('previous_policy_exp_date'),"UTC"),'yyyy-MM-dd'))\
.withColumn('commit_timestamp',from_unixtime(unix_timestamp(col('commit_timestamp')),'yyyy-MM-dd HH:mm:ss'))\
.withColumn('dep_date',date_format(to_utc_timestamp(col('dep_date'),"UTC"),'yyyy-MM-dd'))\
.withColumn('return_date',date_format(to_utc_timestamp(col('return_date'),"UTC"),'yyyy-MM-dd'))\
.withColumn('dat_od_expiry_date',date_format(to_utc_timestamp(col('dat_od_expiry_date'),"UTC"),'yyyy-MM-dd'))\
.withColumn('dat_od_start_date',from_unixtime(unix_timestamp(col('dat_od_start_date')),'yyyy-MM-dd HH:mm:ss'))\
.withColumn('dat_tp_expiry_date',date_format(to_utc_timestamp(col('dat_tp_expiry_date'),"UTC"),'yyyy-MM-dd'))\
.withColumn('dat_tp_start_date',date_format(to_utc_timestamp(col('dat_tp_start_date'),"UTC"),'yyyy-MM-dd'))\
.withColumn('dat_cover_note_date',date_format(to_utc_timestamp(col('dat_cover_note_date'),"UTC"),'yyyy-MM-dd'))

In [ ]:
df_main=df_main.select('reference_num', 'txt_intermediary_cd', 'policy_number', 'certificate_no', 'pol_incept_date', 'pol_exp_date', 'source_system_customer_id', col('sum_insured').cast('String'), col('premiumamount').cast('String'), col('total_premium').cast('String'), 'irda_product_uin', 'product_cd', 'lob', 'renl_cert_no', 'policy_counter', 'business_type', 'txt_business_type', 'commit_timestamp', 'application_no', 'policy_issuance_date', 'rto_location', 'branch', 'prdr_branch_sub_cd', 'customer_gstin', 'policy_type', 'zone', 'previous_policy_no', 'previous_policy_exp_date', 'previous_policy_type', 'policy_effective_from', 'dc_unified_id', 'full_name', 'psnt_address_country', 'psnt_address_state', 'psnt_address_city', 'psnt_address_zip', 'psnt_address_line_1', 'psnt_address_line_2', 'psnt_address_line_3', 'gstin', 'passport', 'pan', 'cleansed_passport', 'cleansed_pan', 'birth_date', 'age', 'agent_name', 'txt_mobile', 'txt_telephone', 'txt_telephone1', 'txt_contact_phone', 'txt_emer_person_mobile_number', 'txt_policy_phone', 'txt_alt_mobile', 'agent_license_no', 'agent_mobile', 'pan_new', 'passport_new', 'address', 'txt_endorsement_wording', 'num_reference_number', col('num_endorsement_no').cast('String'), 'txt_customer_id', col('num_net_od_premium').cast('String'), col('num_net_tp_premium').cast('String'), 'dat_od_expiry_date', 'dat_od_start_date', 'dat_tp_expiry_date', 'dat_tp_start_date', 'num_covernote_no', 'dat_cover_note_date', 'txt_cover_note_hour', 'risk1', 'mot_policy_plan_code', 'mot_policy_plan', 'mot_policy_variant', 'engine_number', 'chassis_number', 'cc', 'yr_of_manf', 'type_of_body', 'registered_seating_capacity', 'trailer_idv', 'owner_occupation', 'totalidvy1', 'totalidvy2', 'totalidvy3', 'nonelectricalaccessoriesy1', 'nonelectricalaccessoriesy2', 'electricalaccessoriesidv', 'veh_idv', 'nonelectricalaccessoriesy3', 'alternatepolicynumber', 'is_cnglpg_kit_pip', 'cng_lpg_si_tot', 'side_car_value', 'roadside_assistance', 'noofclaimsdepreciationreimb', 'voluntary_deductible_amt', 'imposed_excess_amount', 'franchisee', 'compulsory_pa_ownerdriver', 'ncb_misdeclaration', 'vehicle_seg', 'veh_type', 'vehicle_use', 'variant', 'vehicle_age', 'vehcile_subclass', 'vehicle_class', 'policy_plan', 'make', 'model', 'appointeename', 'cum_discount_premium', 'cum_loading_premium', 'endt_discount_premium', 'endt_loading_premium', 'healthcheckup_limit', 'insured_dob', 'insured_name', 'insuredage', 'insuredname', 'loading_reason_amhi', 'member_id', 'memberid', 'nom_dob_hlt', 'nomi_rel_hlt', 'nominee_age', 'nomineename_hlt', 'outpatient_treatmentlimit', 'preexisting_disease', 'relationshipappnom', 'dep_date', 'dob', 'id_no', 'id_type', 'insured_first_name', 'insured_last_name', 'insured_middle_name', 'nominee_name', 'rating_zone', 'relationship', 'return_date', 'travel_duration')

In [ ]:
df_main=df_main.withColumnRenamed('policy_number','policy_no')

In [ ]:
df_main=df_main.withColumn('certificate_no',when((col('certificate_no').isin('0','0.0','/00000','000000','00000000'))|(col('certificate_no').isNull()),lit('00000000')).otherwise(col('certificate_no')))

In [ ]:
try:
     df_main.write.format('org.elasticsearch.spark.sql').mode('append').option('es.index.auto.create', 'true').option('es.nodes' , es_nodes).option('es.port', es_port).option('es.resource', target_index_doc_type).save() 
except Exception as e :
    x = e
else: 
    x = 200 # success
print(x)

In [ ]:
table_name='policy_additional_details'
source='policy_additional_details'
conn_prod = psycopg2.connect(host=prod_host, port=prod_port, user=user_prod, password=pwd_prod, dbname=prod_dbname)
conn_to=conn_prod

In [ ]:
if  x == 200:
    update_progress(table_name,source,time_filter,records,start_time,starttime,'Success')
    print('SUCCESS')
else:
    update_progress(table_name,source,time_filter,'0',start_time,starttime,'Failed')
    print('FAILED')

In [79]:
target_index='get_spa_cover_details_2024-03-20'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
target_index

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'get_spa_cover_details_2024-03-20'

In [81]:
# to add alias to target index - added on 200819
headers = {'Content-Type': 'application/json'}
query_alias= {"actions" : [{ "add" : { "index" : target_index, "alias" : "get_spa_cover_details" }}]}


URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/_aliases'
r = requests.post(URL, data=json.dumps(query_alias), headers=headers)
r

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<Response [200]>

MOT&HLT

In [ ]:
pol.columns

pol=pol.filter(~col('lob').isin('LTA'))

In [ ]:
pol_cover=pol.selectExpr('policy_number as policy_no','certificate_no', 'reference_num','commit_timestamp','lob')

In [ ]:
pol_cover=pol_cover.withColumn('certificate_no',when((col('certificate_no').isin('0','0.0','/00000','000000','00000000'))|(col('certificate_no').isNull()),lit('00000000')).otherwise(col('certificate_no')))\
.withColumn('policy_no',lpad(col('policy_no'),10,'0'))

In [ ]:
from pyspark.sql.window import Window

pol_cover = pol_cover.withColumn("rn", F.row_number()
        .over(Window.partitionBy('policy_no','certificate_no')
        .orderBy(F.col("commit_timestamp").desc(),F.col("reference_num").desc())))

pol_cover = pol_cover.filter(F.col("rn") == 1).drop("rn").drop("primary_key")

In [ ]:

gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark",
"password": "spark@456",
"server.port": "1151-1170",
"dbschema":"public",
"dbtable": "policy_gc_covers_details"}
cover = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.select('reference_num','cover_group','si','premium','loading_premium','discount_premium','total_premium',
'diff_si','cum_total_premium','cum_loading_premium','cum_discount_premium','risk_detail_serial')\
.filter(col('risk_detail_serial')==0).drop_duplicates()

In [ ]:
df_main=pol_cover.join(cover,'reference_num','inner')

In [11]:

gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark",
"password": "spark@456",
"server.port": "1151-1170",
"dbschema":"customermart",
"dbtable": "cover_detail_lta_1"}
cover_lta = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.filter(col('risk_detail_serial')==0).drop_duplicates()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
cover_lta.persist()
cover_lta.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

64004615

In [38]:
df_main=cover_lta.withColumnRenamed('policy_number','policy_no')

df_main=df_main.withColumn('certificate_no',when((col('certificate_no').isin('0','0.0','/00000','000000','00000000'))|(col('certificate_no').isNull()),lit('00000000')).otherwise(col('certificate_no')))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
# from pyspark.sql.window import Window

# df_main = df_main.withColumn("rn", F.row_number()
#         .over(Window.partitionBy('policy_no','certificate_no')
#         .orderBy(F.col("commit_timestamp").desc(),F.col("reference_num").desc())))

# df_main = df_main.filter(F.col("rn") == 1).drop("rn").drop("primary_key")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
report_date = datetime.date.today()- datetime.timedelta(days=2)
report_date_1=report_date
report_date_str = str(report_date)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
report_date_str

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2024-03-19'

In [41]:
target_index=f'get_spa_cover_details_{report_date_str}'
print(target_index)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

get_spa_cover_details_2024-03-19

In [42]:
import json, pprint, requests
es_nodes = '10.35.12.212'
es_port = '9200'
es_user = 'elastic'
es_pwd = 'bEiilauM3es'

######target_index = 'actuarial_premium_risk_tables'----previous name...delete later

#alias_name = 'nepuepr_uat_3'
target_doc_type = f'get_spa_cover_details_{report_date_str}'
#########delete index if already exists########################################################################################################################################################################

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# df_main=pol_cover.select('reference_num').join(cover,'reference_num','inner')

In [ ]:
# df_main=cover.withColumn('primary_key', concat_ws('', trim(col('policy_no')),trim(col('reference_num')),
#                                                        trim(col('certificate_no')))).dropDuplicates()

In [ ]:
# from pyspark.sql.window import Window

# df_main = df_main.withColumn("rn", F.row_number()
#         .over(Window.partitionBy("primary_key")
#         .orderBy(F.col("commit_timestamp").desc())))

# df_main = df_main.filter(F.col("rn") == 1).drop("rn").drop("primary_key")

In [ ]:
df_main.persist()
df_main.count()

In [ ]:
# df_main=df_main.withColumn('report_date',current_date())

In [ ]:
df_main.filter(col('policy_no')=='0237917124').show()

In [72]:
target_index='get_spa_cover_details'
print(target_index)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

get_spa_cover_details

In [73]:
import json, pprint, requests
es_nodes = '10.35.12.212'
es_port = '9200'
es_user = 'elastic'
es_pwd = 'bEiilauM3es'

######target_index = 'actuarial_premium_risk_tables'----previous name...delete later

#alias_name = 'nepuepr_uat_3'
target_doc_type = 'get_spa_cover_details'
#########delete index if already exists########################################################################################################################################################################

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
#########delete index ###############################################
headers = {'Content-Type': 'application/json'}
query = {"query":{"match_all":{}}}
URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/'+target_index
r = requests.delete(URL, data=json.dumps(query), headers=headers) 
print(r)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<Response [200]>

In [71]:
#########create index ##################################################################################################################################################################
headers = {'Content-Type': 'application/json'}
query_create= {"settings": { "index.number_of_shards": "1" }}
URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/'+target_index
r = requests.put(URL, data=json.dumps(query_create), headers=headers)
print(r)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<Response [200]>

In [45]:
target_index_doc_type = target_index +'/'+ target_doc_type

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
target_index_doc_type

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'get_spa_cover_details_2024-03-19/get_spa_cover_details_2024-03-19'

In [49]:
df_main=df_main.drop_duplicates()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
df_main=df_main.withColumn('commit_timestamp',from_unixtime(unix_timestamp(col('commit_timestamp')),'yyyy-MM-dd HH:mm:ss'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
df_main.select('policy_no', 'certificate_no', 'reference_num', 'commit_timestamp', 'lob', 'cover_group', 'currency', 'si', 'premium', 'loading_premium', 'discount_premium', 'total_premium', 'diff_si', 'cum_total_premium', 'cum_loading_premium', 'cum_discount_premium')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[policy_no: string, certificate_no: string, reference_num: bigint, commit_timestamp: string, lob: string, cover_group: string, currency: string, si: decimal(32,2), premium: decimal(32,2), loading_premium: decimal(32,2), discount_premium: decimal(32,2), total_premium: decimal(32,2), diff_si: decimal(32,2), cum_total_premium: decimal(32,2), cum_loading_premium: decimal(32,2), cum_discount_premium: decimal(32,2)]

In [52]:
df_main=df_main.select('reference_num', 'policy_no', 'certificate_no', 'commit_timestamp', 'lob', 'cover_group','currency', col('si').cast('string'), col('premium').cast('string'),
               col('loading_premium').cast('string'), col('discount_premium').cast('string'), col('total_premium').cast('string'), 
               col('diff_si').cast('string'), col('cum_total_premium').cast('string'), col('cum_loading_premium').cast('string'), col('cum_discount_premium').cast('string'),'risk_detail_serial')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
try:
     df_main.write.format('org.elasticsearch.spark.sql').mode('append').option('es.index.auto.create', 'true').option('es.nodes' , es_nodes).option('es.port', es_port).option('es.resource', target_index_doc_type).save() 
except Exception as e :
    x = e
else: 
    x = 200 # success
print(x)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

200

In [ ]:
target_index='get_spa_cover_details_alias'

In [ ]:
# to add alias to target index - added on 200819
headers = {'Content-Type': 'application/json'}
query_alias= {"actions" : [{ "add" : { "index" : target_index, "alias" : "get_spa_cover_details" }}]}


URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/_aliases'
r = requests.post(URL, data=json.dumps(query_alias), headers=headers)
r

In [ ]:
df_main.columns

In [82]:
spark.stop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…